# Zip folder unpacking script
This script will do the following operations in order:
Scan a folder
create new calculation folder
Loop the following actions for each homer script


    Unseal homer template file (.homer -> .zip)
    
    
    Unpack zip file (.zip -> normal directory)
    
    
    Open 'problem' file
    
    
    ####### NOT FOR THIS SCRIPT modify 'problem' file
    
    
    Repack zip file (normal directory -> .zip)
    
    
    Reseal homer file (.zip -> .homer)
    
    
    store updated input file in calculation folder
    
    
    end 

In [18]:
# finding all noteworthy files
import os
import numpy as np 
import zipfile
import shutil
import zlib
import pathlib 
import subprocess
from lxml import etree, objectify
folder = 'D:/repo/homer/ClimateStudies/XPathTesting/HOMERAutomation'
files = [f for f in os.listdir(folder) if f.endswith('.homer')]
n_files = np.size(files)
display(files)

[]

In [19]:
# Unsealing HOMER template files (.homer -> .zip)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.zip')

In [20]:
# Unpack zip file (.zip -> normal directory)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(base + 'TEMP')
    os.remove(file)
# Opening each individiual 'problem' file after extraction
    

In [39]:
# (Within normal directory) Edit the .xml files for each HOMER problem.
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
display(files)
# paths to change as key, new value as value
new_data = {'/HomerProblem/Controllers/Controller/Abbreviation': 'AC', #don't include
            '/HomerProblem/FuelResources/Fuel/Name': 'CCM FUEL',
            '/HomerProblem/FuelResources/Fuel/Name': '',
           ,
           ,}
for i in range(len(files)):
    file = files[i]
    #display(file)
    problem = etree.parse(file + '/problem')
    for j in range(len(new_data))
    #display(problem)
        print(problem.xpath(j)[0].text)
    
#    with open(file + '/problem') as f:
#        xml = f.read()
#       # display(xml)
#       # problem = etree.parse(xml)
#        problem = etree.fromstring(xml)

SyntaxError: invalid syntax (Temp/ipykernel_16744/1659438766.py, line 7)

In [6]:
#Repack zip file (normal directory -> .zip)
## THIS STEP CURRENTLY BREAKS THE HOMER FILE
## Try using zipcode compression 
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
#directory = pathlib.Path('zipfoldertesting/') ## NEEDS TO BE In FOR LOOP, DIRECTORY -> zip file
for i in range(len(files)):
    file = files[i]
    directory = pathlib.Path(file)
    basefile = file.replace('TEMP','')
    subprocess.run(["powershell.exe", "Compress-Archive " + basefile])
    shutil.rmtree(file) #deletes old temp folders
    

['.ipynb_checkpoints',
 'Case1BurlingtonVTDfa.zip',
 'Case1ChicagoDfa.zip',
 'Case1ColumbusOHDfa.zip',
 'Case1DesMoinesIADfa.zip',
 'Case1DetroitMIDfa.zip',
 'Case1LincolnNEDfa.zip',
 'Case1SouthBendINDfa.zip',
 'ZipPackager.ipynb']

In [4]:
# Reseal homer file (.zip -> .homer)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
type(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.homer')